# Test

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from learn import *

In [21]:
sz = 256
num_slice = 9

num_gpus = 2
gpu_start = 0
num_workers = 8
device_ids = range(gpu_start, gpu_start + num_gpus)
torch.cuda.set_device(gpu_start)
bs = 15

## Preprocessing

In [12]:
# datapaths = ['data/train/AOI_2_Vegas_Train', 'data/train/AOI_3_Paris_Train', 'data/train/AOI_4_Shanghai_Train', 'data/train/AOI_5_Khartoum_Train']
# !python code/v5_im-full_rgb.py preproc_test {datapaths[0]}

2018-04-19 15:09:42,598 INFO preproc_test for AOI_2_Vegas
2018-04-19 15:09:42,599 INFO Generate IMAGELIST for inference ... skip
2018-04-19 15:09:42,599 INFO Generate RGB_STORE (test)
2018-04-19 15:09:42,606 INFO prep_rgb_image_store_test for AOI_2_Vegas
2018-04-19 15:09:42,610 INFO Image store file: data/working/images/v5/pred_full_rgb/
2018-04-19 15:34:08,505 INFO preproc_test for AOI_2_Vegas ... done


In [6]:
# !python code/v5_im-full_rgb.py preproc_test {datapaths[3]}

In [7]:
# !python code/v5_im-full_rgb.py preproc_test {datapaths[0]}
# !python code/v5_im-full_rgb.py preproc_test {datapaths[2]}

## Find threshold

In [22]:
model_name = 'deeplab'
data = None

In [23]:
learn, denorm, data = learner_on_dataset(datapaths[0], bs, device_ids, num_workers, model_name='deeplab',
                                         debug=False, global_dataset=True, data=data, num_slice=num_slice, sz=sz,
                                         is_pred=False)
(trn_x,trn_y), (val_x,val_y) = data


Data finished loading: data/train/AOI_3_Paris_Train


## Inferencing

In [24]:
%%time
preds = learn.predict()

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58